In [1]:
import pandas as pd
import sys
sys.path.append("..")
import pandas as pd
from pathlib import Path
import numpy as np
import pickle
from sif_src.glove import load_glove_vectors, calculate_glove_corpus_embeddings
from sif_src.sif import flatten, compute_sif_embeddings_queries, compute_sif_embeddings_texts, compute_word_frequencies, compute_sif_weights, remove_pc_sif
import tqdm
from collections import defaultdict, Counter
from collections.abc import Iterable
from sklearn.decomposition import TruncatedSVD


Calculate and pickle both embeddings for validation set

In [2]:
valid_df = pd.read_pickle("../pickle_backups/marco_valid_df2024-06-04T17.38.1717490321.pickle")



In [3]:
valid_df

,answers,passages,query,query_id,query_type,wellFormedAnswers
0,"[Approximately $15,000 per year.]","{'is_selected': [1, 0, 0, 0, 0, 0], 'passage_t...",walgreens store sales average,9652,numeric,[]
1,"[$21,550 per year, The average hourly wage for...","{'is_selected': [0, 1, 0, 0, 0, 0, 0, 0], 'pas...",how much do bartenders make,9653,numeric,[]
2,"[A boil, also called a furuncle, is a deep fol...","{'is_selected': [0, 0, 0, 0, 0, 0, 1, 0], 'pas...",what is a furuncle boil,9654,description,[]
3,"[Detect and assess a wide range of disorders, ...","{'is_selected': [0, 0, 0, 0, 1, 0, 0, 0, 0], '...",what can urinalysis detect,9655,description,[]
4,"[Shigellosis, diseases of the nervous system, ...","{'is_selected': [0, 0, 0, 0, 1, 0, 0, 0, 0], '...",what is vitamin a used for,9656,description,[]
...,...,...,...,...,...,...
111135,"[It is defined as the quality of being wide, o...","{'is_selected': [0, 0, 0, 0, 0, 1, 0, 0, 0, 0]...",define width,129177,NUMERIC,[]
111136,[Human Resource Management is the term used to...,"{'is_selected': [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]...",forbes definition of human resource management,190655,DESCRIPTION,[]
111137,[No Answer Present.],"{'is_selected': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]...",how long to wax urethane paint,272679,NUMERIC,[]
111138,[No Answer Present.],"{'is_selected': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]...",easiest way to check accounts payable entered ...,177416,DESCRIPTION,[]


In [ ]:
valid_passages = valid_df["passages"]
valid_texts = [passage["passage_text"] for passage in valid_passages]
valid_df["texts"] = valid_texts

#homogenize MS MARCO columns
valid_df['query'] = valid_df['query'].apply(lambda x: [x])

# Load GloVe vectors
glove_vectors = load_glove_vectors('../wv/glove.6B.50d.txt')

In [3]:
queries = valid_df["query"]
texts = valid_df["texts"]
word_freq = compute_word_frequencies(queries + texts)

compute SIF embeddings

In [4]:
sif_weights = compute_sif_weights(word_freq)

In [13]:
query_sif_embeddings = compute_sif_embeddings_queries(queries, glove_vectors, sif_weights)
texts_sif_embeddings = compute_sif_embeddings_texts(texts, glove_vectors, sif_weights)

In [15]:
texts_sif_embeddings = remove_pc_sif(texts_sif_embeddings, alpha=0.001)

In [17]:
valid_df["sif_queries"] = list(query_sif_embeddings)
valid_df["sif_texts"] = list(texts_sif_embeddings)

In [19]:
#queries = valid_df["query"]
#texts = valid_df["texts"]

query_glove_embeddings = calculate_glove_corpus_embeddings(queries, glove_vectors)


In [20]:
texts_glove_embeddings = calculate_glove_corpus_embeddings(texts, glove_vectors)

In [23]:
valid_df["glove_queries"] = list(query_glove_embeddings)
valid_df["glove_texts"] = list(texts_glove_embeddings)

In [30]:
import pickle
from datetime import datetime

TIMESTAMP = datetime.now()
OUTPUT_PATH = Path("../pickle_backups")
TIMESTAMP_STR = TIMESTAMP.isoformat().replace(":", ".")

with open(OUTPUT_PATH / 
    f"valid_embeddings{TIMESTAMP.strftime(r'%Y-%m-%dT%H.%M.%s')}.pickle",
    "wb"
) as f:
    pickle.dump(valid_df, f)